# Benchmark building 

# starting by champs Elysées

In [26]:
import pandas as pd
import numpy as np

In [75]:
champ = pd.read_csv('champ.csv',sep=';')
champ.shape

(9338, 15)

In [42]:
champ.head()

,Identifiant arc,Libelle,Date et heure de comptage,Débit horaire,Taux d'occupation,Etat trafic,Identifiant noeud amont,Libelle noeud amont,Identifiant noeud aval,Libelle noeud aval,Etat arc,Date debut dispo data,Date fin dispo data,geo_point_2d,geo_shape
0,4264,AV_Champs_Elysees,2023-02-01T23:00:00+01:00,764.0,11.28000,Fluide,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.87153587897717, 2.3017227924560615","{""coordinates"": [[2.3009951475338766, 48.87177..."
1,4264,AV_Champs_Elysees,2023-02-01T22:00:00+01:00,819.0,16.52056,Pré-saturé,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.87153587897717, 2.3017227924560615","{""coordinates"": [[2.3009951475338766, 48.87177..."
2,4264,AV_Champs_Elysees,2023-02-01T20:00:00+01:00,1082.0,19.98722,Pré-saturé,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.87153587897717, 2.3017227924560615","{""coordinates"": [[2.3009951475338766, 48.87177..."
3,4264,AV_Champs_Elysees,2023-02-01T18:00:00+01:00,1027.0,23.27334,Pré-saturé,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.87153587897717, 2.3017227924560615","{""coordinates"": [[2.3009951475338766, 48.87177..."
4,4264,AV_Champs_Elysees,2022-12-07T00:00:00+01:00,528.0,68.14334,Bloqué,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.871535879911605, 2.3017227924560624","{""coordinates"": [[2.300995147533877, 48.871777..."


In [76]:
champ['time']=pd.to_datetime(champ['Date et heure de comptage'],utc=True)
champ['debit']=champ['Débit horaire']
champ['taux']=champ['Taux d\'occupation']
champ=champ[['time','debit','taux']]


In [77]:
champ = champ.sort_values(by='time')

In [32]:
champ.isna().sum()

time       0
debit    267
taux     288
dtype: int64

### benchmark simplest model: ignore missing values/ estimate the debit and taux for each weekday as the average of the last same 4 weekdays    

In [78]:
# Convert 'time' column to datetime if it's not already
champ['time'] = pd.to_datetime(champ['time'])

# Set 'time' column as the index
champ.set_index('time', inplace=True)


last_10_days = champ.last('10D')

def calculate_avg_last_4_weeks(df, date, column):
    day_of_week = date.dayofweek
    hour = date.hour
    # Filter for the same day of the week in the last 4 weeks
    past_data = df[(df.index < date) & 
                   (df.index > date - pd.Timedelta(weeks=4)) & 
                   (df.index.dayofweek == day_of_week) & 
                   (df.index.hour == hour)][column]

    # Get the last 4 values and calculate the mean, ignoring NaNs
    return past_data.tail(4).mean(skipna=True)

for date, row in last_10_days.iterrows():
    last_10_days.at[date, 'taux_pred'] = calculate_avg_last_4_weeks(champ, date, 'taux')
    last_10_days.at[date, 'debit_pred'] = calculate_avg_last_4_weeks(champ, date, 'debit')



C:\Users\alkhamed\AppData\Local\Temp\ipykernel_736\3623180710.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_10_days.at[date, 'taux_pred'] = calculate_avg_last_4_weeks(champ, date, 'taux')
C:\Users\alkhamed\AppData\Local\Temp\ipykernel_736\3623180710.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_10_days.at[date, 'debit_pred'] = calculate_avg_last_4_weeks(champ, date, 'debit')


In [80]:
from sklearn.metrics import mean_squared_error
mse_taux = mean_squared_error(last_10_days['taux'], last_10_days['taux_pred'])

mse_debit = mean_squared_error(last_10_days['debit'], last_10_days['debit_pred'])


print(f'MSE_taux : {mse_taux}')
print(f'MSE_debit : {mse_debit}')


MSE_taux : 279.62497252318786
MSE_debit : 60898.26932870371
